<a href="https://colab.research.google.com/github/jihun0203/YOLOv4_pothole/blob/main/PotHole_YOLOv4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 드라이브 연결
- 구글 드라이브에 PotHoleYOLO 폴더로 이동

In [1]:
path = '/content/drive/MyDrive/PotHoleYOLO'

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd /content/drive/MyDrive/PotHoleYOLO

/content/drive/MyDrive/PotHoleYOLO


In [14]:
ls

 darknet/          test/      'training(tiny)'/
 _darknet.labels   training/   training_YOLOv4/


# 환경 설정 (새 프로젝트 실행 시에만)
- 구글 드라이브에 darknet clone

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 6.41 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.
Checking out files: 100% (2050/2050), done.


In [ ]:
cd darknet/

/content/drive/MyDrive/PotHoleYOLO/darknet


- darknet 폴더에 custom 파일 이외의 기존 파일 삭제 (cfg, data ...)

In [ ]:
%cd data/
!find -maxdepth 1 -type f -exec rm -rf {} \;
%cd ..
%rm -rf cfg/
%mkdir cfg

/content/drive/MyDrive/PotHoleYOLO/darknet/data
/content/drive/MyDrive/PotHoleYOLO/darknet


## YOLO 학습 환경 설정
- openCV, GPU, CUDNN, LIBSO 활성화

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

- darknet.py 파일 수정하고 make (그래프 생성 epoch 주기 및 backup 주기) 


In [ ]:
!make

## PotHole Data(Google Drive에서)
### Data 생성
- pothole.data : 클래스 갯수(1개), 클래스 이름 파일 경로 (pothole.names), 백업 경로, train 데이터 목록, test 데이터 목록  
- pothole.names : 클래스 이름 (pothole)  
- pothole 폴더: 사진 이미지 + 같은 이름의 라벨링 txt 파일  

### Data 경로
1. darknet/data 폴더에 생성한 위에서 pothole data 넣기
2. darknet/cfg 폴더에 수정한 cfg 파일 넣기
3. darknet 폴더에 process.py 파일 넣기
4. darknet 폴더에 pretrain 가중치 넣기 (yolov4.conv.137)
5. PotHoleYOLO 폴더 안에 백업할 training 폴더 생성


In [ ]:
# process.py 실행 - 백업 경로 설정 및 train(90%)/test(10%) 데이터 랜덤 목록 생성
!python process.py

/content/drive/MyDrive/PotHoleYOLO/darknet


# YOLO 학습

## Training
### 처음 실행 시:

In [ ]:
!./darknet detector train data/pothole.data cfg/yolov4.cfg yolov4.conv.137 -dont_show -map

### 중간에 런타임 끊겼을 때:

In [ ]:
# 1) Google Drive 재연결
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/PotHoleYOLO/darknet

/content/drive/.shortcut-targets-by-id/1uzw-CCHADpwI4APn81ygaQQaqLW4LYhD/PotHoleYOLO/darknet


In [ ]:
# 권한 부여
!chmod +x ./darknet

* 백업된 마지막 weights 가져와서 다시 학습 (이전의 chart.png는 지워지니 꼭 미리 저장해두고 다시 돌리기!) 

In [ ]:
!./darknet detector train data/pothole.data cfg/yolov4.cfg /content/drive/MyDrive/PotHoleYOLO/training/yolov4_last.weights -dont_show -map

## Test
- cfg 파일의 batch 값 1로 변경
- cfg 파일의 subdivisions 값 1로 변경

In [ ]:
%cd cfg
!sed -i 's/batch=64/batch=1/' yolov4.cfg
!sed -i 's/subdivisions=16/subdivisions=1/' yolov4.cfg
%cd ..

/content/drive/.shortcut-targets-by-id/1uzw-CCHADpwI4APn81ygaQQaqLW4LYhD/PotHoleYOLO/darknet/cfg
/content/drive/.shortcut-targets-by-id/1uzw-CCHADpwI4APn81ygaQQaqLW4LYhD/PotHoleYOLO/darknet


In [ ]:
# 권한 부여
!chmod +x ./darknet

- Best 가중치 가져와서 학습 결과 보여주기

In [ ]:
!./darknet detector map data/pothole.data cfg/yolov4.cfg /content/drive/MyDrive/PotHoleYOLO/training_YOLOv4/yolov4_best.weights

 CUDA-version: 11010 (11020), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 3.2.0
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 32, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    512 x 512 x   3 ->  512 x 512 x  32 0.453 BF
   1 conv     64       3 x 3/ 2    512 x 512 x  32 ->  256 x 256 x  64 2.416 BF
   2 conv     64       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  64 0.537 BF
   3 route  1 		                           ->  256 x 256 x  64 
   4 conv     64       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  64 0.537 BF
   5 conv     32       1 x 1/ 1    256 x 256 x  64 ->  256 x 256 x  32 0.268 BF
   6 conv     64       3 x 3/ 1    256 x 256 x  32 ->  256 x 256 x  64 2.416 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 256 x 256 x  64 0.004 BF
   8 conv 